In [69]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import precision_recall_curve, auc, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta
from xgboost import XGBClassifier

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [70]:
train_df = pd.read_csv('obs_train.csv')
test_df = pd.read_csv('obs_test.csv')

print(train_df.shape, test_df.shape)
train_df.info(verbose=True)

(331637, 263) (130483, 262)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331637 entries, 0 to 331636
Data columns (total 263 columns):
 #    Column                                                    Dtype  
---   ------                                                    -----  
 0    person_id                                                 int64  
 1    SepsisLabel                                               int64  
 2    datetime                                                  object 
 3    datetime_year                                             int64  
 4    datetime_month                                            int64  
 5    datetime_day                                              int64  
 6    datetime_hour                                             int64  
 7    datetime_weekday                                          int64  
 8    MALE                                                      float64
 9    age_in_months                                             floa

In [71]:
def make_predictions(train_df, test_df):
    train_df = train_df.drop(columns=['person_id', 'datetime'])

    test_labels = test_df[['person_id', 'datetime']].copy()
    test_df = test_df.drop(columns=['person_id', 'datetime'])


    train_y = train_df['SepsisLabel']
    train_X = train_df.drop(columns='SepsisLabel')
    test_X = test_df

    mmx = MinMaxScaler()
    train_X = mmx.fit_transform(train_X)
    test_X = mmx.transform(test_X)

    clf = XGBClassifier(
                colsample_bytree = 0.7501745945565902,
                gamma = 0.39428938023558957,
                learning_rate = 0.03908010701819611,
                max_depth = 4,
                min_child_weight = 4.8769931443216805,
                n_estimators = 300,
                reg_alpha = 0.21536773880455967,
                reg_lambda = 0.15388032812481203,
                subsample = 0.7923260274694592,
                objective='binary:logistic',
                eval_metric='aucpr',
                random_state=0,
            )

    clf.fit(train_X, train_y)

    test_proba_y = clf.predict_proba(test_X)[:, 1]
    test_labels['SepsisLabel'] = test_proba_y
    test_labels['person_id_datetime'] = test_labels['person_id'].astype(str) + '_' + test_labels['datetime'].astype(str)
    test_labels[['person_id_datetime', 'SepsisLabel']].to_csv('preds.csv', index = False)


    train_proba_y = clf.predict_proba(train_X)[:, 1]

    train_prec, train_recall, _ = precision_recall_curve(train_y, train_proba_y)
    train_pr_auc = auc(train_recall, train_prec)

    print(train_pr_auc)

In [72]:
make_predictions(train_df=train_df, test_df=test_df)

0.9114576085595291


In [73]:
import kaggle

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

api.competition_submit()